### Does the pipeline builder allow us to pickle correctly?

In [5]:
import luigi

import dill

from experiment_framework import helpers
from empirical_privacy.one_bit_sum import GenSampleOneBitSum

In [2]:
n = 40
p = 0.5
ds_rs = {
        'dataset_settings': {
            'n_trials'      : n,
            'prob_success': p,
            'gen_distr_type': 'binom',
        },
        'random_seed'     : '1338',
    }
ccc_kwargs = {
        'n_trials_per_training_set_size': 10,
        'n_max'                         : 64,
        'dataset_settings'              : ds_rs['dataset_settings'],
        'validation_set_size'           : 10
        }

In [6]:
%%time
CCC = helpers.build_convergence_curve_pipeline(GenSampleOneBitSum,
                                           gensample_kwargs={
                                               'generate_in_batch': True
                                           })
CCC2_inst = CCC(**ccc_kwargs)
luigi.build([CCC2_inst], local_scheduler=True, workers=8, log_level='ERROR')

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 38.7 ms


### Does AllAsymptotics cache correctly?


In [3]:
All = helpers.AllAsymptotics(
        gen_sample_path='empirical_privacy.one_bit_sum.GenSampleOneBitSum',
        dataset_settings=ds_rs['dataset_settings'])

In [4]:
AA = All.requires()[0]

In [6]:
with AA.output().open() as f:
    res = dill.load(f)

In [7]:
print(res)

{'mean': 0.52070724699345583, 'median': 0.52096700509437244, 'std': 0.014188469748888202, 'n_bootstraps': 5757, 'p': 0.99, 'upper_bound': 0.53070724699345584}


In [8]:
%%time
luigi.build([All], local_scheduler=True, workers=8, log_level='ERROR')

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 41.9 ms


/opt/conda/lib/python3.6/site-packages/luigi/parameter.py:261: UserWarning: Parameter "task_process_context" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))


True

Yes, it completes instantlly rather than in 4 minutes.

### What happens with empty kwargs?

In [17]:
kwargs = {}
import inspect

def f(a=0, b=1, c=2):
    return (a+2*b+3*c)
    
print(inspect.getcallargs(f, **kwargs))

{'a': 0, 'b': 1, 'c': 2}


###### Nothing -- good